# Testing the Integrated Simulation Platform
This notebook contains codes to test the various functionalities of the integrated simulation platform. You can call any of the modules from the package here. 

In [6]:
#Go to the main directory
#!pip install --editable .
#!pip install pandas
# !pip install wntr
# !pip install pandapower
# !pip install sklearn

#to find requirements
#depfinder -y .

In [7]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Import required packages

In [8]:
from pathlib import Path
from infrarisk.src.network_recovery import *
import infrarisk.src.simulation as simulation
from infrarisk.src.network_sim_models.integrated_network import *

from infrarisk.src.network_sim_models.interdependencies import *

from infrarisk.src.optimizer import *
import infrarisk.src.plots as model_plots

import random

## Create an IntegratedNetwork object

In [9]:
simple_network = IntegratedNetwork(name = "Micropolis")

### Load the three infrastructure models: Water, Power and Transportation

Three different models are used:
- Water distribution network using **wntr** package
- Power systems using **pandapower** package
- Transportation network using static traffic assignment package developed by Dr. Stephen Boyles (University of Texas at Austin)

In [10]:
MAIN_DIR = Path('..')

network_dir= 'in2'
water_file = MAIN_DIR/f'infrarisk/data/networks/{network_dir}/water/water.inp'
power_file = MAIN_DIR/f'infrarisk/data/networks/{network_dir}/power/power.json'
transp_folder = MAIN_DIR/f'infrarisk/data/networks/{network_dir}/transportation/'

# load all infrastructure networks
simple_network.load_networks(water_file, power_file, transp_folder, '1ph')

Water network successfully loaded from ..\dreaminsg_integrated_model\data\networks\in2\water\water.inp. The analysis type is set to Pressure Dependent Demand Analysis.
initial simulation duration: 60s; hydraulic time step: 60s; pattern time step: 3600s

Power system successfully loaded from ..\dreaminsg_integrated_model\data\networks\in2\power\power.json. Single phase power flow simulation will be used.

Transportation network successfully loaded from ..\dreaminsg_integrated_model\data\networks\in2\transportation. Static traffic assignment method will be used to calculate travel times.


### Create a Networkx graph of the integrated infrastructure network.

In [113]:
simple_network.generate_integrated_graph()

Successfully added power network to the integrated graph...
Successfully added water network to the integrated graph...
Successfully added transportation network to the integrated graph...
Integrated graph successffully created.


Loading BokehJS ...

### Build interdependencies

Three types of dependencies:
- Power - Water dependencies (eg.: water pump on electric motor, generator on reservoir)
- Power - Transportation dependencies (eg.: road access to power system components for M&R)
- Water - Transportation dependencies (eg.: road access to water network components for M&R)

The dependencies are referenced using two tables in the model.
- **wp_table** for water - power dependencies
- **access_table** for transportation dependencies

In [14]:
dependency_file = MAIN_DIR/f"infrarisk/data/networks/{network_dir}/dependecies.csv"
simple_network.generate_dependency_table(dependency_file = dependency_file)
simple_network.dependency_table.wp_table

,water_id,power_id,water_type,power_type
0,W_WP9,P_MP1,Pump,Motor
1,W_R9,P_G3,Reservoir,Generator


In [15]:
simple_network.dependency_table.access_table.head()

,origin_id,transp_id,origin_cat,origin_type,access_dist
0,P_B1,T_J2,power,Bus,180.28
1,P_B2,T_J5,power,Bus,111.80
2,P_B3,T_J5,power,Bus,141.42
3,P_B4,T_J8,power,Bus,100.00
4,P_B5,T_J7,power,Bus,100.00


### Set failed components

In [16]:
scenario_file = MAIN_DIR/"infrarisk/data/disruptive_scenarios/{}/{}.csv".format('test1', 'motor_failure_net1')

simple_network.set_disrupted_components(scenario_file=scenario_file)
simple_network.get_disrupted_components()

['T_L2', 'P_MP1', 'W_P21', 'W_WP9']

In [17]:
simple_network.pipe_leak_node_generator()

### Set initial crew locations

In [18]:
simple_network.set_init_crew_locs(init_power_loc='T_J8',
init_water_loc='T_J8', init_transpo_loc='T_J8')

## Simulation of interdependent effects using a test scenario
### (a) Create NetworkRecovery

In [30]:
network_recovery = NetworkRecovery(simple_network, sim_step=60)

### (b) Create a simulation object

In [31]:
sim_step = (
    simple_network.wn.options.time.hydraulic_timestep
)  # initial_sim_step which will be updated during the simulation
bf_simulation = simulation.NetworkSimulation(network_recovery, sim_step)

### (c) Generation of random repair order

In [32]:
#Generate a random repair order
repair_order = network_recovery.network.get_disrupted_components()
random.shuffle(repair_order)
print('Current repair order is {}'.format(repair_order))

Current repair order is ['P_MP1', 'T_L2', 'W_WP9', 'W_P21']


### (d) Generation of event tables

In [33]:
bf_simulation.network_recovery.schedule_recovery(repair_order)

Iteration 1: gap 4642.279088: time 0.000000
Iteration 2: gap 0.000936: time 0.007248
Iteration 3: gap 0.000000: time 0.010365
P_MP1 ('power', 'MP', 'motor', 'Motor')
Iteration 1: gap 0.354227: time 0.001037
Iteration 2: gap 0.000936: time 0.004148
Iteration 3: gap 0.000000: time 0.009502


In [34]:
bf_simulation.expand_event_table(20)

### (e) Simulation of interdependent effects

In [35]:
resilience_metrics = bf_simulation.simulate_interdependent_effects(
    bf_simulation.network_recovery)

resilience_metrics.set_weighted_auc_metrics()


Simulating network conditions at 60 s
Simulation time:  60.0 ; Hydraulic time step:  60.0 ; Report time step:  60
['control 1', 'control 2']
Supply:  0.06939921609791994 Base demand:  0.06939921604

Simulating network conditions at 4740 s
Simulation time:  4740.0 ; Hydraulic time step:  60.0 ; Report time step:  4740
['control 1', 'control 2']
Supply:  0.06939921609917374 Base demand:  0.06939921604

Simulating network conditions at 6060 s
Simulation time:  6060.0 ; Hydraulic time step:  60.0 ; Report time step:  6060
['control 1', 'control 2']
Supply:  0.06939921609951889 Base demand:  0.06939921604

Simulating network conditions at 6120 s
Simulation time:  6120.0 ; Hydraulic time step:  60.0 ; Report time step:  6120
['control 1', 'control 2']
Supply:  0.06939921609801265 Base demand:  0.06939921604

Simulating network conditions at 6180 s
Simulation time:  6180.0 ; Hydraulic time step:  60.0 ; Report time step:  6180
['control 1', 'control 2', 'W_WP9_power_off_6120.0', 'W_WP9_power

### (f) Calculation of resilience metric

In [36]:
# auc = 0.5 * water_auc + 0.5 * power_auc
print(
    "Water AUC: ",
    round(resilience_metrics.water_auc, 3),
    "\t",
    "Power AUC: ",
    round(resilience_metrics.power_auc, 3),
    "\t",
    "Weighted AUC: ",
    round(resilience_metrics.weighed_auc, 3),
)

Water AUC:  0.127 	 Power AUC:  0.965 	 Weighted AUC:  0.546


### Plot the direct and interdependent effects

In [111]:
#Plotting repair curves
model_plots.plot_repair_curves(network_recovery, scatter = False)

In [112]:
time_tracker, power_consump_tracker, water_consump_tracker = (
    resilience_metrics.time_tracker, 
    resilience_metrics.power_consump_tracker, 
    resilience_metrics.water_consump_tracker
    )

model_plots.plot_interdependent_effects(
    time_tracker,
    power_consump_tracker,
    water_consump_tracker,
    scatter = False)